In [1]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install spotipy

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score
from sklearn.metrics import silhouette_score
from sklearn.metrics import classification_report
from sklearn.metrics import pairwise_distances_argmin_min

In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="e5af6275806e4e8ebf14a5d71e202fad",
                                                           client_secret="a72673ed83bb437d83c801b760a066d5"))

In [102]:
def get_df_from_playlist(playlist_id):
    
    results = sp.user_playlist_tracks("spotify", playlist_id)
    tracks = results['items']

    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    track_ids = [track['track']['uri'] for track in tracks]
    
    start = 0
    df = pd.DataFrame()

    for stop in range(0, len(track_ids)+100, 100):
        if start != stop:
            new_df = pd.json_normalize(sp.audio_features(track_ids[start:stop]))
            df = pd.concat([new_df, df])
            start = stop

    df = df.reset_index(drop=True)
    df.drop(['type', 'uri', 'track_href', 'analysis_url',], axis=1, inplace=True)
    
    main_artist_names = [track['track']['artists'][0]['name'] for track in tracks]
    song_names = [track['track']['name'] for track in tracks]
    df['main_artist'] = main_artist_names
    df['song_name'] = song_names

    return df

In [38]:
def get_df_from_multi_playlists(playlists):

    df = pd.DataFrame()

    for pl in playlists:

        df_ = get_df_from_playlist(playlists[pl])
        df = pd.concat([df,df_]).drop_duplicates().reset_index(drop=True)

    return df

In [103]:
#official Billboard Hot 100 playlist

df_hot100_current = get_df_from_playlist("6UeSakyzhiEt4NB3UAd6NQ")

In [ ]:
#From Wikipedia: List of Billboard Hot 100 top-ten singles in 2023

#r = requests.get('https://en.wikipedia.org/wiki/List_of_Billboard_Hot_100_top-ten_singles_in_2023')
#r.status_code

In [42]:
#official Spotify top 50 charts: Global and top 10 markets
#https://en.wikipedia.org/wiki/List_of_largest_recorded_music_markets (2022 table)

top_by_market_playlists = {
    
    "top_gb" : "37i9dQZEVXbNG2KDcFcKOF", #Global
    "top_us" : "37i9dQZEVXbLp5XoPON0wI", #United States
    "top_jp" : "37i9dQZEVXbKqiTGXuCOsB", #Japan
    "top_uk" : "37i9dQZEVXbMwmF30ppw50", #United Kingdom
    "top_de" : "37i9dQZEVXbK8BKKMArIyl", #Germany
    "top_fr" : "37i9dQZEVXbIPWwFssbupI", #France
    "top_sk" : "37i9dQZEVXbJZGli0rRP3r", #South Korea
    "top_ca" : "37i9dQZEVXbMda2apknTqH", #Canada
    "top_br" : "37i9dQZEVXbKzoK95AbRy9", #Brazil
    "top_au" : "37i9dQZEVXbK4fwx2r07XW", #Australia
    "top_it" : "37i9dQZEVXbJUPkgaWZcWG" #Italy (no playlist available for China)
}

In [104]:
df_top_all_markets = get_df_from_multi_playlists(top_by_market_playlists)

In [105]:
df_top_songs = pd.concat([df_hot100_current, df_top_all_markets]).drop_duplicates().reset_index(drop=True)

In [106]:
df_top_songs

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,main_artist,song_name
0,0.868,0.538,5.0,-8.603,1.0,0.1740,0.269,0.000003,0.0901,0.732,99.968,2IGMVunIBsBLtEQyoI1Mu7,231750.0,4.0,Doja Cat,Paint The Town Red
1,0.429,0.453,0.0,-7.746,1.0,0.0459,0.554,0.000002,0.1020,0.155,77.639,4KULAymBBJcPRpk1yO4dOG,227196.0,4.0,Zach Bryan,I Remember Everything (feat. Kacey Musgraves)
2,0.712,0.603,8.0,-5.520,1.0,0.0262,0.186,0.000000,0.1150,0.670,97.994,1Lo0QY9cvc8sUB2vnIOxDT,265493.0,4.0,Luke Combs,Fast Car
3,0.552,0.702,9.0,-5.707,1.0,0.1570,0.117,0.000021,0.1050,0.564,169.994,1BxfuPKGuaTgP7aM0Bbdwr,178427.0,4.0,Taylor Swift,Cruel Summer
4,0.492,0.675,6.0,-5.456,1.0,0.0389,0.467,0.000000,0.1420,0.478,203.759,7K3BhSpAxZBznislvUMVtn,163855.0,4.0,Morgan Wallen,Last Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,0.740,0.856,6.0,-5.385,0.0,0.1270,0.122,0.000115,0.1840,0.467,134.957,5BoXWVomF4aOi0VL2G71iF,139099.0,4.0,Vale Pain,In Mezzo ai Guai (feat. Neima Ezza)
438,0.716,0.737,6.0,-5.831,0.0,0.0578,0.115,0.000000,0.0922,0.584,131.947,4midGjEpVz0mYkIdQMFX3l,179864.0,4.0,Geolier,CHIAGNE (feat. Lazza & Takagi & Ketra)
439,0.802,0.712,9.0,-7.204,0.0,0.4230,0.139,0.000000,0.0750,0.515,101.896,3UYr4TdhtCthMH9ujhx0BA,133859.0,4.0,Rhove,Pelé
440,0.458,0.648,6.0,-8.372,1.0,0.0974,0.228,0.000000,0.0892,0.234,80.835,3myPe2jZvDbPoNXY7v2TU0,181853.0,3.0,Tedua,Malamente


In [ ]:
#most popular music genres
#https://headphonesaddict.com/music-genre-statistics/#Most_Popular_Music_Genre_Statistics
#separate by genres and decades
#top_genres_gb = ['Pop', 'Rock', '90s', '80s', 'Soundtracks', 'Hip-hop/Rap/Trap', 'R&B', 'Dance/Electronic/House', '70s', 'Soul/Blues']

In [ ]:
#most popular music genres
#spotify playlists

#"most streamed rap songs on spotify"

In [68]:
#best of playlists
#genre
best_by_genre_playlists = {
    
    "best_hiphop" : "37i9dQZF1DX5qmTk3PVatJ", #greatest hiphop beats of all time - 50
    "best_classic_rock" : "37i9dQZF1DWXRqgorJj26U", #rock classics - 200
    "best_pop" : "7AsZxdanrHJFNGp3q7tp1f" #best of pop 1990 2023 - 1200
}

In [107]:
df_best_by_genre = get_df_from_multi_playlists(best_by_genre_playlists)

In [71]:
# decade
best_by_decade_playlists = {
    
    "best_80s" : "37i9dQZF1DX4UtSsGT1Sbe", #all out 80s - 50
    "best_90s" : "37i9dQZF1DXbTxeAdrVG2l" #all out 90s - 50
}

In [108]:
df_best_by_decade = get_df_from_multi_playlists(best_by_decade_playlists)

In [109]:
#concat all songs
df_all_best_songs = pd.concat([df_best_by_genre, df_best_by_decade]).drop_duplicates().reset_index(drop=True)

In [110]:
df_all_best_songs

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,main_artist,song_name
0,0.816,0.775,11,-3.323,1,0.2340,0.160000,0.000310,0.0543,0.527,93.431,503OTo2dSqe7qk76rgsbep,270587,4,Dr. Dre,Still D.R.E.
1,0.763,0.786,10,-6.472,0,0.2290,0.014600,0.011400,0.0817,0.504,93.857,33ZXjLCpiINn8eQIDYEPTD,325507,4,Mobb Deep,"Shook Ones, Pt. II"
2,0.592,0.770,1,-3.402,1,0.3180,0.122000,0.000001,0.1030,0.361,191.692,3XrvEifl0hIzgBGUa5jBLS,264760,4,Clipse,Grindin'
3,0.831,0.682,7,-5.697,1,0.2380,0.256000,0.000000,0.4940,0.600,91.771,2KFg8kjYy0rYjWsXGhEQHN,319360,4,The Notorious B.I.G.,Who Shot Ya? - 2005 Remaster
4,0.674,0.695,6,-8.636,0,0.2780,0.038700,0.002020,0.1940,0.773,151.486,6ScJMrlpiLfZUGtWp4QIVt,221840,4,Lil Wayne,A Milli
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1819,0.556,0.920,8,-4.606,1,0.0367,0.002710,0.000210,0.1170,0.771,121.138,3mwvKOyMmG77zZRunnxp9E,159227,4,Diddy,I'll Be Missing You (feat. Faith Evans & 112) ...
1820,0.725,0.583,1,-9.942,0,0.0427,0.300000,0.000238,0.0665,0.762,79.984,3yUcJwYu7fXAfqMj9krY6l,218360,4,Oasis,Don't Look Back In Anger - Remastered
1821,0.452,0.909,9,-3.977,1,0.0366,0.031300,0.060300,0.2300,0.805,124.623,1v2zyAJrChw5JnfafSkwkJ,225787,4,Deep Blue Something,Breakfast At Tiffany's
1822,0.619,0.670,11,-10.322,0,0.0353,0.001650,0.010900,0.3310,0.777,129.269,45Ia1U4KtIjAPPU7Wv1Sea,211933,4,Aerosmith,Cryin'


In [58]:
#df_all_best_filtered = pd.merge(df_all_best_songs, df_top_songs, how='outer', indicator=True)
#df_all_best_filtered = df_all_best_filtered[df_all_best_filtered['_merge']=='left_only']

## II

In [155]:
df_all = df_all_best_songs.copy()

In [156]:
x = df_all[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

In [157]:
scaler = StandardScaler()
x_prep = scaler.fit_transform(x)

In [158]:
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(x_prep)

clusters = kmeans.predict(x_prep)

C:\Users\grwc9\AppData\Roaming\Python\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [159]:
scaled_df = pd.DataFrame(x_prep, columns=x.columns)
scaled_df['song_name'] = df_all['song_name']
scaled_df['main_artist'] = df_all['main_artist']
scaled_df['cluster'] = clusters
scaled_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song_name,main_artist,cluster
0,1.025566,0.210330,1.552459,1.170072,0.860499,1.835033,0.286881,-0.291962,-0.885438,-0.453170,-1.143221,Still D.R.E.,Dr. Dre,1
1,0.645839,0.282379,1.280182,-0.000316,-1.162117,1.773410,-0.659598,-0.215677,-0.702129,-0.559824,-1.125387,"Shook Ones, Pt. II",Mobb Deep,1
2,-0.579315,0.177580,-1.170315,1.140710,0.860499,2.870307,0.039521,-0.294086,-0.559630,-1.222934,2.970175,Grindin',Clipse,1
3,1.133035,-0.398813,0.463349,0.287728,0.860499,1.884332,0.911792,-0.294094,2.056204,-0.114659,-1.212711,Who Shot Ya? - 2005 Remaster,The Notorious B.I.G.,1
4,0.008186,-0.313664,0.191072,-0.804610,-1.162117,2.377320,-0.502720,-0.280199,0.049170,0.687566,1.287074,A Milli,Lil Wayne,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1819,-0.837242,1.160069,0.735627,0.693219,0.860499,-0.596628,-0.736996,-0.292649,-0.465968,0.678292,0.016648,I'll Be Missing You (feat. Faith Evans & 112) ...,Diddy,1
1820,0.373583,-1.047255,-1.170315,-1.290011,-1.162117,-0.522680,1.198209,-0.292457,-0.803819,0.636557,-1.706138,Don't Look Back In Anger - Remastered,Oasis,2
1821,-1.582365,1.088019,1.007904,0.927000,0.860499,-0.597860,-0.550890,0.120691,0.290015,0.835954,0.162537,Breakfast At Tiffany's,Deep Blue Something,1
1822,-0.385870,-0.477412,1.552459,-1.431245,-1.162117,-0.613882,-0.743896,-0.219116,0.965716,0.706114,0.357028,Cryin',Aerosmith,2


In [160]:
scaled_df.groupby(['cluster', 'main_artist'], as_index=False).count().sort_values(['cluster', 'key'], ascending=[True, False])[['main_artist', 'cluster', 'key']].reset_index(drop=True)

,main_artist,cluster,key
0,Calvin Harris,0,3
1,Aerosmith,0,2
2,Lady Gaga,0,2
3,P!nk,0,2
4,Rihanna,0,2
...,...,...,...
1125,Yes,2,1
1126,Zara Larsson,2,1
1127,a-ha,2,1
1128,blink-182,2,1


## III

In [111]:

#format: "song title" OR "song tile" + "artist"

In [164]:
def get_song_by_keywords(kws):
    
    results = sp.search(q=f'track:{kws}', limit=1)
    
    track_id = results['tracks']['items'][0]['id']
    track_name = results['tracks']['items'][0]['name']
    track_artist = results['tracks']['items'][0]['artists'][0]['name']
    
    is_top_song = track_id in df_top_songs['id'].values
    
    print(track_name, '-', track_artist)
    
    audio_features = sp.audio_features(track_id)
    df_ = pd.DataFrame(audio_features)
    new_features = df_[x.columns]
    scaled_x = scaler.transform(new_features)
    cluster = kmeans.predict(scaled_x)
    
    return cluster, scaled_x, is_top_song

In [165]:
#kws = input('Choose a song: ')
kws = 'fast car'
cluster, scaled_x, is_top_song = get_song_by_keywords(kws)
cluster, scaled_x, is_top_song

Fast Car - Luke Combs


(array([2]),
 array([[ 0.28044245, -0.91625638,  0.73562688,  0.35351324,  0.8604987 ,
         -0.72603698,  0.45612771, -0.29409399, -0.47934843,  0.20994077,
         -0.9522045 ]]),
 True)

In [166]:
if (song_chosen[1]):
    print('yay')
else:
    print('nay')

yay


In [167]:
filtered_df = np.array(scaled_df[scaled_df['cluster'] == cluster[0]][x.columns], order="C")
closest, _ = pairwise_distances_argmin_min(scaled_x, filtered_df)
print('\n[RECOMMENDED SONG]')
print(' - '.join([scaled_df.loc[closest]['song_name'].values[0], scaled_df.loc[closest]['main_artist'].values[0]]))


[RECOMMENDED SONG]
The Sweet Escape - Gwen Stefani
